In [ ]:
#ライブラリをインポート
import csv
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
#マクロを定義
BINS = 10000  #ヒストグラムのビンの数
EPSILON = .00001  #スムージングパラメータ
UPPER_LIMIT = 1.1 #静止区間の上限
LOWER_LIMIT = 0.9 #静止区間の加減
STATIONARY_INTERVALS = 5  #静止区間除去のサンプルの間隔(何サンプル静止区間が連続したら除去するか)

In [ ]:
#加速度データのCSVファイルから3軸加速度を取り出す関数
def retrieve_acceleration(filename):
    accX = []
    accY = []
    accZ = []
    #ｘ軸,y軸,z軸の値を配列に格納
    with open(filename) as f:
        reader = csv.reader(f)
        for row in reader:
            accX.append(float(row[1]))
            accY.append(float(row[2]))
            accZ.append(float(row[3]))
    return accX, accY, accZ

In [ ]:
#静止区間除去関数
def remove_stationary_intervals(accX, accY, accZ):

    #各軸の加速度の平均を求める
    accAveX = sum(accX) / len(accX)
    accAveY = sum(accY) / len(accY)
    accAveZ = sum(accZ) / len(accZ)

    #重力加速度の推定値=合成加速度の平均を求める
    accResAve = math.sqrt(accAveX ** 2 + accAveY ** 2 + accAveZ ** 2)

    #各時刻の合成加速度を求める
    accRes = [math.sqrt(x ** 2 + y ** 2 + z ** 2) for x, y, z in zip(accX, accY, accZ)]

    #各時刻の合成加速度から静止区間(重力加速度の推定値に近い値が一定以上以上連続している区間)を除去する
    i=0 #ループ変数
    counter = 0 #静止区間がSTATIONARY_INTERVALS分続いているかをカウントする変数
    while i < len(accRes):
        if accResAve * LOWER_LIMIT < accRes[i] < accResAve * UPPER_LIMIT:   #平均のLOWER_LIMIT倍~UPPER_LIMIT倍の範囲を調べる
            counter += 1    #範囲内ならカウントを増やす
            if counter == STATIONARY_INTERVALS: #カウントがSTATIONARY_INTERVALSに達したらその区間を削除
                del accRes[i+1-STATIONARY_INTERVALS:i+1]    #スライスでは選択範囲の開始位置startと終了位置stopを[start:stop]のように書く。start <= x < stopの範囲が選択される。start番目の値は含まれるがstop番目の値は含まれない。
                counter = 0
                i -= STATIONARY_INTERVALS   #削除した分インデックスがズレるので補正する
        else:
            counter = 0
        i += 1

    return accRes   #静止区間を除去した後のリストを返す

In [ ]:
#連続する2サンプルの差分を取る関数
def differences_of_acceleration(accRes):
    index = 0
    accDif = []
    for dif in accRes[:-1]:
        accDif.append(math.fabs(accRes[index + 1]*100000 - accRes[index]*100000))
        index += 1

    return accDif

In [ ]:
#KLダイバージェンス関数
def KL_divergence(a, b):
    min_value = min(min(a), min(b)) #a,bの最小値の小さい方
    max_value = max(max(a), max(b)) #a,bの最大値の大きい方

    #a,bのヒストグラムを作成し、同じ数のビンで区切る
    a_hist, _ = np.histogram(a, bins=BINS, range=(min_value, max_value))
    b_hist, _ = np.histogram(b, bins=BINS, range=(min_value, max_value))

    #正規化する(確率分布に変換する、合計を1にする)ために全合計で割る
    a_hist = (a_hist + EPSILON) / a_hist.sum()
    b_hist = (b_hist + EPSILON) / b_hist.sum()

    #KLダイバージェンスの値を返す
    return np.sum([ai * np.log(ai / bi) for ai, bi in zip(a_hist, b_hist)])

In [ ]:
#ファイルを読み込む
number = 1046
filename1 = "walk100Hz-0809-0900.csv".format(number)
filename2 = "walk100Hz-0809-1000.csv".format(number)

In [ ]:
accX1, accY1, accZ1 = retrieve_acceleration(filename1)
accX2, accY2, accZ2 = retrieve_acceleration(filename2)

In [ ]:
print(accX1[:5])
print(accY1[:5])
print(accZ1[:5])

In [ ]:
print(sum(accX1))
print(sum(accY1))
print(sum(accZ1))

In [ ]:
accAveX1 = sum(accX1) / len(accX1)
accAveY1 = sum(accY1) / len(accY1)
accAveZ1 = sum(accZ1) / len(accZ1)

#重力加速度の推定値=合成加速度の平均を求める
accResAve1 = math.sqrt(accAveX1 ** 2 + accAveY1 ** 2 + accAveZ1 ** 2)

print(accAveX1, accAveY1, accAveZ1, accResAve1)

In [ ]:
#各時刻の合成加速度を求める
accRes1 = [math.sqrt(x ** 2 + y ** 2 + z ** 2) for x, y, z in zip(accX1, accY1, accZ1)]

print(accRes1[:5])
len(accRes1)

In [ ]:
print(accRes1[:5])
len(accRes1)

In [ ]:
i=0 #ループ変数
counter = 0 #静止区間がSTATIONARY_INTERVALS分続いているかをカウントする変数
while i < len(accRes1):
    if accResAve1 * LOWER_LIMIT < accRes1[i] < accResAve1 * UPPER_LIMIT:   #平均のLOWER_LIMIT倍~UPPER_LIMIT倍の範囲を調べる
        counter += 1    #範囲内ならカウントを増やす
        if counter == STATIONARY_INTERVALS: #カウントがSTATIONARY_INTERVALSに達したらその区間を削除
            del accRes1[i-4:i+1]    #スライスでは選択範囲の開始位置startと終了位置stopを[start:stop]のように書く。start <= x < stopの範囲が選択される。start番目の値は含まれるがstop番目の値は含まれない。
            counter = 0
            i -= 5   #削除した分インデックスがズレるので補正する
    else:
        counter = 0
    i += 1

print(i, accRes1[:5], len(accRes1))

In [ ]:
accDif1 = differences_of_acceleration(acc1)
accDif2 = differences_of_acceleration(acc2)

In [ ]:
i=0
counter = 0
while i < len(acc):
    if accAve * 0.9 < acc[i] < accAve * 1.1:
        counter += 1
        if counter == 5:
            del acc[i-4:i+1]
            counter = 0
            i -= 5
    else:
        counter = 0
    i += 1

In [ ]:
acc=[1.0, 1.4, 0.6, 0.91, 1.09, 1.08, 0.92, 1.05, 0.95, 1.3, 0.7]
accAve = sum(acc) / len(acc)
accAve

In [ ]:
start = None
end = None
counter = 0
for i in range(len(acc)):
    if accAve * 0.9 <= acc[i] <= accAve * 1.1:
        if start is None:
            start = i
        end = i
        counter += 1
        if counter == 5:
            if start is not None and end is not None:
                del acc[start:end+1]
                start = None
                end = None
                counter = 0
                i -= 5
    else:
        start = None
        end = None
        counter = 0

print(acc)

In [ ]:
accAve = sum(acc) / len(acc)
start = None
end = None
counter = 0
while True:
    for i in range(len(acc)):
        if accAve * 0.9 <= acc[i] <= accAve * 1.1:
            if start is None:
                start = i
            end = i
            counter += 1
            if counter == 5:
                if start is not None and end is not None:
                    del acc[start:end+1]
                    start = None
                    end = None
                    counter = 0
                    break
        else:
            start = None
            end = None
            counter = 0
    else:
        break
    
acc

In [110]:
accRes=[1.0, 1.4, 0.6, 0.91, 1.09, 1.08, 0.92, 1.05, 0.95, 1.3, 0.7]
accResAve = sum(accRes) / len(accRes)
accResAve

1.0

In [113]:
i=0 #ループ変数
counter = 0 #静止区間がSTATIONARY_INTERVALS分続いているかをカウントする変数
while i < len(accRes):
    if accRes[i] < accResAve * 1.1 and accRes[i] > accResAve * 0.9:   #平均のLOWER_LIMIT倍~UPPER_LIMIT倍の範囲を調べる
        counter += 1    #範囲内ならカウントを増やす
        if counter == 5: #カウントがSTATIONARY_INTERVALSに達したらその区間を削除
            del accRes[i-4:i+1]    #スライスでは選択範囲の開始位置startと終了位置stopを[start:stop]のように書く。start <= x < stopの範囲が選択される。start番目の値は含まれるがstop番目の値は含まれない。
            counter = 0
            i -= 5   #削除した分インデックスがズレるので補正する
        else:
            counter = 0
        i += 1

KeyboardInterrupt: 

In [1]:
accRes=[1.0, 1.4, 0.6, 0.91, 1.09, 1.08, 0.92, 1.05, 0.95, 1.3, 0.7]
accResAve = sum(accRes) / len(accRes)

start = None
end = None
counter = 0
while True:
    for i in range(len(accRes)):
        if accResAve * 0.9 <= accRes[i] <= accResAve * 1.1:
            if start is None:
                start = i
            end = i
            counter += 1
            if counter == 5:
                if start is not None and end is not None:
                    del accRes[start:end+1]
                    start = None
                    end = None
                    counter = 0
                    break
        else:
            start = None
            end = None
            counter = 0
    else:
        break

print(accRes)


[1.0, 1.4, 0.6, 0.95, 1.3, 0.7]


In [2]:
accRes=[1.0, 1.4, 0.6, 0.91, 1.09, 1.08, 0.92, 1.05, 0.95, 1.3, 0.7]
accResAve = sum(accRes) / len(accRes)

index = 0
n = 0
for stop in accRes:
    if accResAve * 0.9 <= stop <= accResAve * 1.1:
        index += 1
    else:
        index = 0
    if index == 5:
        del accRes[n-4:n]
        index = 0
    n += 1

print(accRes)

[1.0, 1.4, 0.6, 1.05, 0.95, 1.3, 0.7]
